In [4]:
from urllib.parse import quote

query_url = 'https://yts-subs.com/search/fight%20club'

query = 'fight club'

print(quote(query))

# Quote is to encode text into a valid url format

fight%20club


In [2]:
from requests_html import HTMLSession


website_url = "https://yts-subs.com/"

def url_to_response(url):
    session = HTMLSession()
    response = session.get(url)
    return response

response = url_to_response(website_url)
print(response)

<Response [200]>


In [27]:
from urllib.parse import quote

website_url = "https://yts-subs.com/"
query = "fight club"
query_encoded = quote(query)
website_search_url = website_url + "search/" + query_encoded

print(website_search_url)

response = url_to_response(website_search_url)
print(response)

https://yts-subs.com/search/fight%20club
<Response [200]>


In [30]:
search_results = response.html.find('.media-movie-clickable')

if len(search_results) > 0:
    print(search_results)
else:
    print("No results found.")

[<Element 'li' class=('media', 'media-movie-clickable')>, <Element 'li' class=('media', 'media-movie-clickable')>]


In [69]:
single_search_result = search_results[0]

def search_result_to_movie_info(single_search_result):
    poster = single_search_result.find('.media-object', first=True).attrs["src"]
    title = single_search_result.find(".media-heading", first=True).text

    year_raw = single_search_result.find(".movinfo-section")[0].text
    year = int(year_raw.split()[0])
    movie_page_raw = single_search_result.find(".media-body", first=True).find("a", first=True)
    movie_page = movie_page_raw.absolute_links.pop()

    movie_info = {"title":title,
                 "year": year,
                 "poster": poster,
                 "movie_page": movie_page}

    return movie_info

movie_info_list = list(map(search_result_to_movie_info, search_results))

print(movie_info_list)

[{'title': 'Fight Club', 'year': 1999, 'poster': 'https://www.yifysubtitles.com/images/movies-small/movie-1603.jpg', 'movie_page': 'https://yts-subs.com/movie-imdb/tt0137523'}, {'title': 'Zombie Fight Club', 'year': 2014, 'poster': 'https://www.yifysubtitles.com/images/movies-small/movie-4753.jpg', 'movie_page': 'https://yts-subs.com/movie-imdb/tt3560742'}]


In [72]:
def select_movie_number(movie_info_list):
    for i in range(len(movie_info_list)):
        single_movie_info = movie_info_list[i]
        print(f'{i+1} : {single_movie_info["title"]} - {single_movie_info["year"]}')

    selected_movie = int(input("Enter the movie number: "))
    return movie_info_list[selected_movie - 1]
              
selected_movie = select_movie_number(movie_info_list)

1 : Fight Club - 1999
2 : Zombie Fight Club - 2014
Enter the movie number: 1


In [74]:
response = url_to_response(selected_movie["movie_page"])
print(response)

<Response [200]>


In [92]:
subtitle_list = response.html.find('.high-rating')

def subtitle_html_to_dict(single_subtitle):
    subtitle_language = single_subtitle.find(".sub-lang", first=True).text

    download_cell = single_subtitle.find('.download-cell', first=True)
    download_page_link = download_cell.find("a", first=True).absolute_links.pop()

    subtitle_rating = int(single_subtitle.find(".rating-cell", first=True).text)

    subtitle_dict = {"language": subtitle_language,
                    "download_page": download_page_link,
                    "rating": subtitle_rating}

    return subtitle_dict


subtitle_info_list = list(map(subtitle_html_to_dict, subtitle_list))

print(subtitle_info_list)

[{'language': 'Arabic', 'download_page': 'https://yts-subs.com/subtitles/fight-club-arabic-yify-119474', 'rating': 0}, {'language': 'Arabic', 'download_page': 'https://yts-subs.com/subtitles/fight-club-arabic-yify-131544', 'rating': 0}, {'language': 'Arabic', 'download_page': 'https://yts-subs.com/subtitles/fight-club-arabic-yify-2120', 'rating': 0}, {'language': 'Bengali', 'download_page': 'https://yts-subs.com/subtitles/fight-club-bengali-yify-93844', 'rating': 0}, {'language': 'Bengali', 'download_page': 'https://yts-subs.com/subtitles/fight-club-bengali-yify-110523', 'rating': 0}, {'language': 'Brazilian portuguese', 'download_page': 'https://yts-subs.com/subtitles/fight-club-brazilian-portuguese-yify-9920', 'rating': 2}, {'language': 'Brazilian portuguese', 'download_page': 'https://yts-subs.com/subtitles/fight-club-brazilian-portuguese-yify-99244', 'rating': 0}, {'language': 'Bulgarian', 'download_page': 'https://yts-subs.com/subtitles/fight-club-bulgarian-yify-6319', 'rating': 0

In [117]:

def filter_subtitles(subtitle_info_list, filter_language="English"):
    results = []
    for single_subtitle in subtitle_info_list:
        if single_subtitle['language'] == filter_language:
            results.append(single_subtitle)

    if len(results) == 0:
        selected_subtitle = None
    else:
        selected_subtitle = results[0]

    return selected_subtitle

final_subtitle = filter_subtitles(subtitle_info_list)
#filtered_subtitle = filter_subtitles(subtitle_info_list, filter_language="Wakanda")

In [119]:

def download_page_to_zip_url(final_subtitle):
    response = url_to_response(final_subtitle["download_page"])
    download_link = response.html.find('.download-subtitle', first=True).attrs["href"]
    return download_link

zip_url = download_page_to_zip_url(final_subtitle)

In [125]:
import requests

def download_zip_file(zip_url):
    zip_file_data = requests.get(zip_url).content
    filename = zip_url.split("/")[-1]

    with open(filename, "wb") as zip_file:
        zip_file.write(zip_file_data)
    
    return 0

download_zip_file(zip_url)

fight-club-english-yify-16425.zip
